## 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd

import os

import warnings
warnings.filterwarnings('ignore')

path = '/opt/ml/input/data/train/'

train = pd.read_csv(path + 'train_ratings.csv')

year_data = pd.read_csv(os.path.join(path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(path, 'directors.tsv'), sep='\t')

## 베이스라인 시작 전 해야하는 train_new 파일 및 item2attributes 생성 코드

In [3]:
# 모델 시작 전 인코딩하는 과정입니다.
user2idx = {user:idx for idx, user in enumerate(train['user'].unique())}
idx2user = {idx:user for idx, user in enumerate(train['user'].unique())}

## 패딩 때문에 0 비워놓음
item2idx = {item:(idx+1) for idx, item in enumerate(train['item'].unique())}
idx2item = {(idx+1):item for idx, item in enumerate(train['item'].unique())}

genre2idx = {genre:(idx) for idx, genre in enumerate(genre_data['genre'].unique())}
idx2genre = {(idx):genre for idx, genre in enumerate(genre_data['genre'].unique())}

train_new = train.copy()
genre_data_new = genre_data.copy()

train_new['user'] = train_new['user'].map(user2idx)
train_new['item'] = train_new['item'].map(item2idx)

genre_data_new['item'] = genre_data_new['item'].map(item2idx)
genre_data_new['genre'] = genre_data_new['genre'].map(genre2idx)
train_new.to_csv(os.path.join(path, 'train_new.csv'), index = False)
genre_data_new.to_csv(os.path.join(path, 'genre_data_new.csv'), index = False)

import json
genre_dic = genre_data_new.groupby('item')['genre'].apply(list).to_dict()

with open(os.path.join(path, 'item2attributes.json'),'w') as f:
    json.dump(genre_dic, f)#, indent=4)

## 모델 학습 후 submission 값 디코딩하는 코드

In [7]:
# 모델 학습 후 디코딩하는 과정입니다.
out_path = './output'
output = pd.read_csv(os.path.join(out_path, 'submission.csv'))
output['user'] = output['user'].map(idx2user)
output['item'] = output['item'].map(idx2item)
output.to_csv(os.path.join(out_path, 'deco_trainAG_submission.csv'), index = False)

## 앙상블 하는 코드

In [54]:
# 베이스라인 저장된 csv 호출
baseline = pd.read_csv(os.path.join(out_path, 'deco_trainAG_submission.csv'))
# ease 모델 저장된 csv 호출
ease = pd.read_csv(os.path.join(out_path, 'ease.csv'))

In [55]:
# tem : seq 만들어주기 위한 도구
# seq : 랭킹을 매기기 위한 값. 낮을 수록 더 유망한 것.
baseline['tem'] = 2
ease['tem'] = 2
baseline['seq'] = baseline.groupby('user')['tem'].apply(lambda x : x.cumsum())
ease['seq'] = ease.groupby('user')['tem'].apply(lambda x : x.cumsum())
ease['seq'] = ease['seq'] - 1 # ease를 한 단계 높게 처주기 위해.
baseline['seq'] = baseline['seq'] # (+ 2) : 10개 중 baseline 4개만 반영하기 위해 +2, +4 등 조치 취함.

In [56]:
# 베이스라인과 ease 합침.
final = pd.concat([baseline, ease])
# 베이스라인과 ease에서 모두 추천하는 영화 찾기 위한 코드.
# 두 모델에서 모두 추천하는 영화는 0순위로 놓기로 함
final['seq'][final.duplicated(['user','item'], False)] = 0 # 0 : 0순위.
# 이후 중복 제거
final = final.drop_duplicates(['user','item'])
# 유저 단위로, seq가 낮을 수록 더 높은 순위에 추천이기 때문에 이렇게 함.
final = final.sort_values(['user','seq']).reset_index(drop = True)
# 상위 10개만 추림
final = final.groupby('user').apply(lambda x : x[:10]).reset_index(drop = True)
final[['user','item']].to_csv(os.path.join(out_path, 'ensemble2.csv'), index = False)